# SI 618 WN 2018 - Homework 4: Using the Spark RDD API to analyze bigrams in text

### <font color="red">NOTE: If your PySpark installation isn't working please plan to attend office hours or set up an appointment with Kai (Mac or Windows) or Dr. Teplovs (Mac) as soon as possible.

## Objectives
1. To gain familiarity with PySpark
2. To learn the basics of the Spark RDD API
3. To get experience finding and downloading data

## Please fill in...
### * Your name: Akio Kakishima
### * People you worked with:  I worked by myself

## Submission Instructions:
Please turn in this Jupyter notebook file (in both .ipynb and .html formats) **and the text file that contains the text you analyzed** via Canvas.

## Overview

This project is designed to give you a basic familiarity with the Apache Spark RDD API.

**Your first task** is to run the next code cell in this notebook, as is, (without modification) and confirm that everything is working.

Your second and main task is to modify the word_count_v2.ipynb file in Lab 4 to create a si618-hw4-YOUR_UNIQNAME, which counts the number of bigrams within the corpus. At the conclusion of its execution, it should output three pieces of information: 
1. the total number of bigrams
2. the 20 most common bigrams
3. the minimum number of bigrams required to add up to 10% of all bigrams

In [1]:
import re
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

totc = ["It was the best of times",
    "it was the worst of times",
    "it was the age of wisdom",
    "it was the age of foolishness",
    "it was the epoch of belief",
    "it was the epoch of incredulity",
    "it was the season of Light",
    "it was the season of Darkness",
    "it was the spring of hope",
    "it was the winter of despair",
    "we had everything before us",
    "we had nothing before us",
    "we were all going direct to Heaven",
    "we were all going direct the other way"]

WORD_RE = re.compile(r"\b[\w']+\b")

input_text = sc.parallelize(totc)
word_counts_sorted = input_text.flatMap(lambda line: WORD_RE.findall(line)) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda accumulator, value: accumulator + value) \
    .sortBy(lambda x: x[1], ascending = False)

top10_sorted = word_counts_sorted.take(10)
top10_sorted

[('the', 11),
 ('was', 10),
 ('of', 10),
 ('it', 9),
 ('we', 4),
 ('us', 2),
 ('age', 2),
 ('season', 2),
 ('epoch', 2),
 ('before', 2)]

## Bigram Analysis
Bigrams are pairs of continguous words.  For example, consider the text "The quick brown fox".  The bigrams in that text are: (The, quick), (quick, brown), (brown,fox).

You should be able to use the above code as a starting point, with the main difference being that you will be extracting pairs of words rather than single words.

Here are the steps you need to include:

1. Find and download a text file from the Gutenberg Project.  Please select a book written in English and download the plain text version (i.e. the .txt file).  
1. Normalize the text by converting it to lowercase.  
1. Extract all bigrams from the text.  Don’t try to get fancy: just take all the pairs of words from each line of your text file
1. Perform a mapping to yield a count of one for each bigram (e.g. (("the", "quick"), 1)). 
1. Reduce this list of bigrams with count of one to counts of bigrams (e.g. (("the", "quick"), 312)).
1. Sort to determine the most frequent and total number of bigrams. 
1. Report the total number of bigrams, the top 20 most common bigrams and the minimum number of bigrams required to add up to 10% of all bigrams.


### Step 1a: Find and download a text file
Go to Project Gutenberg (http://www.gutenberg.org) and find a book written in English that you think sounds interesting. 
Download the plain text (.txt) version of the book.

### Step 1b: Load the text file into Spark

In [2]:
crime_punishment_text = "crime_punishment.txt"
crime_punishment = sc.textFile(crime_punishment_text)

### Steps 2 and 3: Normalize text and extract bigrams
Note: there are many ways to accomplish this.  We recommend you create
a function that both creates bigrams and normalizes text.  The following
template code assumes this is the approach you are planning to take.

In [3]:
def some_function(line_taken):
    list_words = []
    list_words = WORD_RE.findall(line_taken.lower())

    counter = 0
    list_bigrams = []
    for idx, each_word in enumerate(list_words):
        if idx == 0:
            pass
        else:
            bigram = (list_words[idx-1],each_word)   # THIS HAS TO BE A TUPLE
            list_bigrams.append(bigram)
    return list_bigrams
    
bigrams1 = crime_punishment.flatMap(lambda line: some_function(line))
# Debug: make sure it's a list of bigrams
temp_output = sc.parallelize(bigrams1.take(10))
temp_output.collect()

[('the', 'project'),
 ('project', 'gutenberg'),
 ('gutenberg', 'ebook'),
 ('ebook', 'of'),
 ('of', 'crime'),
 ('crime', 'and'),
 ('and', 'punishment'),
 ('punishment', 'by'),
 ('by', 'fyodor'),
 ('fyodor', 'dostoevsky')]

### Step 4:  Map the bigrams to key-value pairs where the key is the bigram and the value is 1

In [4]:
bigrams2 = bigrams1.map(lambda each_bigram: (each_bigram, 1))

# Debug: make sure it's a list of bigrams
temp_output = sc.parallelize(bigrams2.take(10))
temp_output.collect()

[(('the', 'project'), 1),
 (('project', 'gutenberg'), 1),
 (('gutenberg', 'ebook'), 1),
 (('ebook', 'of'), 1),
 (('of', 'crime'), 1),
 (('crime', 'and'), 1),
 (('and', 'punishment'), 1),
 (('punishment', 'by'), 1),
 (('by', 'fyodor'), 1),
 (('fyodor', 'dostoevsky'), 1)]

### Step 5: Reduce the (word,1) key-value pairs to give you counts of each bigram

In [5]:
bigrams3 = bigrams2.reduceByKey(lambda a, b: a + b)

### Step 6: Sort the resulting RDD by value in descending order

In [6]:
bigrams_sorted = bigrams3.sortBy(lambda x: x[1], ascending = False)

### Step 7: Report the required values
1. the total number of bigrams
2. the 20 most common bigrams
3. the minimum number of bigrams required to add up to 10% of all bigrams

In [7]:
#1. the total number of bigrams
total_bigrams = bigrams_sorted.count()
print("Total number of bigrams: {}".format(total_bigrams))

Total number of bigrams: 77794


In [8]:
#2. the 20 most common bigrams
top20 = sc.parallelize(bigrams_sorted.take(20))
top20.collect()

[(('in', 'the'), 757),
 (('of', 'the'), 585),
 (('he', 'was'), 481),
 (('to', 'the'), 481),
 (('don', 't'), 464),
 (('he', 'had'), 460),
 (('on', 'the'), 458),
 (('it', 's'), 451),
 (('i', 'am'), 441),
 (('at', 'the'), 440),
 (('it', 'was'), 399),
 (('that', 's'), 355),
 (('that', 'he'), 323),
 (('you', 'are'), 310),
 (('to', 'be'), 301),
 (('in', 'a'), 301),
 (('do', 'you'), 282),
 (('with', 'a'), 253),
 (('did', 'not'), 248),
 (('for', 'the'), 241)]

In [9]:
#3. the minimum number of bigrams required to add up to 10% of all bigrams
bigram_list = sc.parallelize(bigrams_sorted.take(10000))
percentage = 0.1
accum_total = 0
i_list = []
for i in bigram_list.collect():
    if accum_total < (total_bigrams*percentage):
        accum_total += i[1]
        i_list.append(i)
    else: pass
print("Minimum number of bigrams required to add up to 10% of all bigrams: {}".format(len(i_list)))

Minimum number of bigrams required to add up to 10% of all bigrams: 19


### Above and Beyond
Crime and Punishment is the story of the protagonist, Raskolnikov (also known as Rodia). It tells the story of his anguish, guilt and glee of murdering a pawnbroker. People play an important role in his well being in the book. For example, the police officer in charge of investigating the death will cause negative impact on Rodia, while the thoughts of Rodia's sister will mostly bring him joy. 

In this kernel, I will analyze the frequency of a Names that tells a snapshot of how often each character is addresssed. I will filter out some basic words that may not point to character names that will still be caught by the regex. Pronouns are kept in, as they may be important in showing what sort of people are addressed (whether it is a man or woman, or self, other...)
This deserves extra credit because I will be reanalyzing the text, this time with a single word, as opposed to a bigram. 

To no surprise, Raskolnikov pops up as the top. Interestingly, Svidrigaïlov shows up as second-- he is a character known to symbolize human desire, and generally bad vibes. Svidrigaïlov is followed by Rodia's sister, Dunia.


Changes

-Regex now filters through for words that are capitalized

-Filter words that are not relevant

In [37]:
above_re = re.compile(r"\b^[A-Z][\w]+\b")
#junk_list = ["And", "But", "You","The","What","Why","How","She","That"]
word_counts_sorted = crime_punishment.flatMap(lambda line: above_re.findall(line)) \
                        .map(lambda word: (word, 1)) \
                        .reduceByKey(lambda a, b: a + b) \
                        .sortBy(lambda x: x[1], ascending = False) \
                        .filter(lambda x: "And" not in x[0]) \
                        .filter(lambda x: "But" not in x[0]) \
                        .filter(lambda x: "The" not in x[0]) \
                        .filter(lambda x: "A" not in x[0]) \
                        .filter(lambda x: "When" not in x[0]) \
                        .filter(lambda x: "Where" not in x[0]) \
                        .filter(lambda x: "Who" not in x[0]) \
                        .filter(lambda x: "How" not in x[0]) \
                        .filter(lambda x: "That" not in x[0]) \
                        .filter(lambda x: "This" not in x[0]) \
                        .filter(lambda x: "So" not in x[0]) \
                        .filter(lambda x: "In" not in x[0]) \
                        .filter(lambda x: "For" not in x[0]) \
                        .filter(lambda x: "At" not in x[0])                 
#for i in junk_list:
#    word_counts_sorted.filter(lambda x: i not in x[0])
top20_sorted = sc.parallelize(word_counts_sorted.take(20))
top20_sorted.collect()

[('He', 267),
 ('Raskolnikov', 208),
 ('She', 57),
 ('Svidrigaïlov', 50),
 ('Dounia', 42),
 ('Ivanovna', 41),
 ('Razumihin', 38),
 ('Petrovitch', 35),
 ('It', 33),
 ('Katerina', 33),
 ('Porfiry', 28),
 ('His', 28),
 ('Pyotr', 25),
 ('You', 24),
 ('Zametov', 18),
 ('Pulcheria', 16),
 ('Gutenberg', 16),
 ('Lebeziatnikov', 15),
 ('Marmeladov', 14),
 ('Romanovna', 13)]